# Entrenamiento Personalizado: StyleGan2-ADA

En este cuaderno, realizaremos transfer learning con StyleGAN2 y conjuntos de datos personalizados.

Esto significa que no entrenaremos la red GAN desde cero con nuestras imágenes (ya que toma alrededor de dos semanas), sino que usaremos el modelo ya entrenado en otras imágenes como punto de partida. Esto reducirá el tiempo de entrenamiento a unas horas al omitir las etapas iniciales donde la red neuronal aprende características de bajo nivel de imágenes que son muy similares para cualquier tipo de imágenes.

In [ ]:
#@title 1. Montar Google Drive
#@markdown Accede a tu Google Drive para cargar imágenes desde bases de datos o modelos pre-entrenados y guardar los resultados.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 2. Instalación de Librerías
#@markdown StyleGAN2-ADA se instalará en tu Google Drive para acelerar el proceso de entrenamiento.

#@markdown Ejecuta esta celda. Si ya has instalado el repositorio, omitirá el proceso de instalación y actualizará el directorio del repositorio. Si no lo has instalado, instalará todos los archivos necesarios.
#@markdown Puedes notar algunos errores: Ignóralos por ahora, son problemas de compatibilidad que no afectan nuestro trabajo.
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/angelv-salazar/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/angelv-salazar/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install torch==1.8.1 torchvision==0.9.1
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install scipy==1.10.1
!pip install ninja

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

In [ ]:
#@title 3. Comprimamos nuestro dataset en un archivo zip.

#@markdown El archivo que recibe StyleGan2 es un zip que contiene nuestras imágenes. Este paso lo debemos hacer solo la primera vez que entrenamos nuestra base de datos.

#@markdown Escribe el directorio de tu conjunto de datos.

#@markdown **RECUERDA:** La ruta de la imagen no debe contener espacios " ", para evitar errores, usa un símbolo como "_" en su lugar.
input_dir = '/content/drive/MyDrive/data/gan01/images' #@param {type: "string"}
#@markdown Escribe la ruta y coloca un nombre al archivo *zip* donde se almacenará el *dataset* convertido. Debes crearlo la primera vez.
dataset_file = '/content/drive/MyDrive/data/gan01/verify.zip' #@param {type: "string"}

if not dataset_file.endswith('.zip'):
  dataset_file += '.zip'

!python dataset_tool.py --source {input_dir} --dest {dataset_file}

In [ ]:
#@title 4. Proceso de entrenamiento de Modelo StyleGAN2 Personalizado

#@markdown Ingresa la ruta del archivo *zip* del *dataset* creado en la celda anterior.
dataset = "/content/drive/MyDrive/data/gan01/verify.zip" #@param {type: "string"}

#@markdown Para el aprendizaje por transferencia, establece los puntos de partida **ffhq512** o **ffhq1024** según la resolución de tus imágenes. Esto debes hacerlo solo la primera vez.
#@markdown Si deseas reanudar el proceso de entrenamiento, omite el archivo zip y proporciona la ruta a tu último archivo *.pkl* que se encuentra en la ruta: **colab-sg2-ada-pytorch/stylegan-ada-pytorch/results** de tu Drive.
resume_from = "ffhq512" #@param {type: "string"}

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap=1 --cfg='11gb-gpu' --data={dataset} --aug=noaug --mirror=False --mirrory=False --metrics=None --resume={resume_from}

### Mientras estás entrenando...
¡Una vez que la celda anterior esté en ejecución, debería estar entrenando!

¡No cierres esta pestaña! Colab debe estar abierta y operativa para continuar con el entrenamiento.

Cada 40 minutos aproximadamente, se debería agregar una nueva línea a tu salida, indicando que aún estás entrenando. Dependiendo de la configuración de snapshot_count, deberías ver que la carpeta de resultados (/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results) en tu carpeta de Google Drive se llena tanto con muestras (fakesXXXXXx.jpg) como con pesos del modelo (model weights) (network-snapshot-XXXXXX.pkl). Vale la pena revisar las muestras durante el entrenamiento, pero no te preocupes demasiado por cada muestra individual.

Una vez que Colab se haya apagado (esto es frecuente debido a sus limites), puedes volver a reanudar el entrenamiento. Para esto debes ejecutar de nuevo algunas celdas del cuaderno: Montar Drive, Instalar Librerias y actualizar la ruta de tu último archivo .pkl en *resume_from* para continuar el entrenamiento desde el modelo más reciente.